In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 1s (200 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
51 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as reposito

In [27]:
# Import necessary libraries
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("WineClustering").getOrCreate()

# Load the dataset
file_path = "/content/drive/MyDrive/ColabNotebooks/datasets/wine-clustering.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Preprocess the data by dropping any rows with null values
df = df.dropna()

In [28]:
# Assemble features
from pyspark.ml.feature import VectorAssembler
feature_columns = df.columns  # Assuming all columns are features, adjust as necessary
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(df)

# Set different values of k for the KMeans algorithm
k_values = [2, 10, 15, 20]
results = {}

In [29]:
# Perform KMeans clustering for each k and evaluate using Silhouette score
for k in k_values:
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(df)

    # Make predictions
    predictions = model.transform(df)

    # Evaluate clustering by computing Silhouette score
    evaluator = ClusteringEvaluator()
    silhouette_score = evaluator.evaluate(predictions)
    results[k] = silhouette_score

    # Print results
    print(f"For k = {k}, Silhouette Score = {silhouette_score}")

For k = 2, Silhouette Score = 0.8213603513331723
For k = 10, Silhouette Score = 0.6810238482437947
For k = 15, Silhouette Score = 0.6844672306874956
For k = 20, Silhouette Score = 0.6393344257672275


In [9]:
# Stop the Spark session
spark.stop()